In [1]:
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from measure_optimize import MeasurementOptimizer
import matplotlib.pyplot as plt
from itertools import permutations, product
from scipy.sparse import coo_matrix
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxModel
from pyomo.contrib.pynumero.interfaces.external_grey_box import ExternalGreyBoxBlock

## Input Setting

In [2]:
jaco_info = pd.read_csv('./RotaryBed/Q22_scale.csv', index_col=False)

jaco_list = np.asarray(jaco_info)

jaco = []
for i in range(len(jaco_list)):
    jaco.append(list(jaco_list[i][1:]))
    
print(np.shape(jaco))

(308, 5)


In [3]:
number_measure = 14
number_t = 22
number_total = number_measure*number_t
cost = [1]*number_total

## FIM collection and Sigma INV

In [4]:
solution1 = [1]*number_total

In [5]:
calculator = MeasurementOptimizer(jaco, number_measure, number_t, cost, verbose=True)

fim_expect = calculator.fim_computation()

calculator.compute_FIM(solution1)

Q shape: 308 5
======FIM result======
FIM: [[ 1.12672079e+03  1.99620709e+00 -8.27109148e+02  2.23307383e+03
  -1.27981839e+02]
 [ 1.99620709e+00  1.18589056e+00 -2.94047895e-01  1.63272938e+01
  -3.30259408e+00]
 [-8.27109148e+02 -2.94047895e-01  6.68554142e+02 -1.72703074e+03
   3.26205209e+01]
 [ 2.23307383e+03  1.63272938e+01 -1.72703074e+03  5.20782946e+03
  -2.12490410e+02]
 [-1.27981839e+02 -3.30259408e+00  3.26205209e+01 -2.12490410e+02
   1.16854209e+02]]
Determinant: 1435753590.9147868 ; log_e(det): 21.084955698778014 ; log_10(det): 9.157079911153815
Trace: 7121.144491083985 ; log_e(trace): 8.870823734618002 ; log_10(trace): 3.852549797880995
Min eig: 0.8358678444970318 ; log_e(min_eig): -0.17928475913869069 ; log_10(min_eig): -0.07786238158328697
Cond: 8134.8284627660305


array([[ 1.12672079e+03,  1.99620709e+00, -8.27109148e+02,
         2.23307383e+03, -1.27981839e+02],
       [ 1.99620709e+00,  1.18589056e+00, -2.94047895e-01,
         1.63272938e+01, -3.30259408e+00],
       [-8.27109148e+02, -2.94047895e-01,  6.68554142e+02,
        -1.72703074e+03,  3.26205209e+01],
       [ 2.23307383e+03,  1.63272938e+01, -1.72703074e+03,
         5.20782946e+03, -2.12490410e+02],
       [-1.27981839e+02, -3.30259408e+00,  3.26205209e+01,
        -2.12490410e+02,  1.16854209e+02]])

In [6]:
class LogDetModel(ExternalGreyBoxModel):
    def __init__(self, use_exact_derivatives=True,verbose=False):
        self._use_exact_derivatives = use_exact_derivatives
        self.verbose = verbose

        # For use with exact Hessian
        self._output_con_mult_values = np.zeros(1)

        if not use_exact_derivatives:
            raise NotImplementedError("use_exact_derivatives == False not supported")
        
    def input_names(self):
        return ['a', 'b', 'c', 'e', 'g',
               'h', 'j', 'k', 'm', 'n',
               'o', 'p', 'q', 'r', 's']

    def equality_constraint_names(self):
        # no equality constraints
        return [ ]
    
    def output_names(self):
        return ['log_det']

    def set_output_constraint_multipliers(self, output_con_multiplier_values):
        assert len(output_con_multiplier_values) == 1
        np.copyto(self._output_con_mult_values, output_con_multiplier_values)

    def finalize_block_construction(self, pyomo_block):
        # initialize, set up LB and UB
        for i, ele in enumerate(elements):
            pyomo_block.inputs[ele].setlb(initial[i])
            pyomo_block.inputs[ele].setub(initial[i])
        
        

    def set_input_values(self, input_values):
        self._input_values = list(input_values)

    def evaluate_equality_constraints(self):

        # Not sure what this function should return with no equality constraints
        return None
    
    def evaluate_outputs(self):
        a = self._input_values[0]
        b = self._input_values[1]
        c = self._input_values[2]
        e = self._input_values[3]
        g = self._input_values[4]
        
        h = self._input_values[5]
        j = self._input_values[6]
        k = self._input_values[7]
        m = self._input_values[8]
        n = self._input_values[9] 
        
        o = self._input_values[10]
        p = self._input_values[11]
        q = self._input_values[12]
        r = self._input_values[13]
        s = self._input_values[14]

        # form matrix
        M = np.array([[a,h,j,k,m],
                      [h,b,n,o,p],
                     [j,n,c,q,r],
                     [k,o,q,e,s],
                     [m,p,r,s,g]])

        # compute log determinant
        (sign, logdet) = np.linalg.slogdet(M)

        if self.verbose:
            print("\n Consider M =\n",M)
            print("   logdet = ",logdet,"\n")

        return np.asarray([logdet], dtype=np.float64)

    def evaluate_jacobian_equality_constraints(self):
        return None

    def evaluate_jacobian_outputs(self):

        a = self._input_values[0]
        b = self._input_values[1]
        c = self._input_values[2]
        e = self._input_values[3]
        g = self._input_values[4]
        
        h = self._input_values[5]
        j = self._input_values[6]
        k = self._input_values[7]
        m = self._input_values[8]
        n = self._input_values[9] 
        
        o = self._input_values[10]
        p = self._input_values[11]
        q = self._input_values[12]
        r = self._input_values[13]
        s = self._input_values[14]

        if self._use_exact_derivatives:

            M = np.array([[a,h,j,k,m],
                      [h,b,n,o,p],
                     [j,n,c,q,r],
                     [k,o,q,e,s],
                     [m,p,r,s,g]])

            # compute inverse
            Minv = np.linalg.pinv(M)

            row = np.zeros(15)
            col = np.zeros(15)
            data = np.zeros(15)
            row[0], col[0], data[0] = (0, 0, Minv[0,0])
            row[1], col[1], data[1] = (0, 1, Minv[1,1])
            row[2], col[2], data[2] = (0, 2, Minv[2,2])
            row[3], col[3], data[3] = (0, 3, Minv[3,3])
            row[4], col[4], data[4] = (0, 4, Minv[4,4])
            row[5], col[5], data[5] = (0, 5, 2*Minv[0,1])
            row[6], col[6], data[6] = (0, 6, 2*Minv[0,2])
            row[7], col[7], data[7] = (0, 7, 2*Minv[0,3])
            row[8], col[8], data[8] = (0, 8, 2*Minv[0,4])
            row[9], col[9], data[9] = (0, 9, 2*Minv[1,2])
            row[10], col[10], data[10] = (0, 10, 2*Minv[1,3])
            row[11], col[11], data[11] = (0, 11, 2*Minv[1,4])
            row[12], col[12], data[12] = (0, 12, 2*Minv[2,3])
            row[13], col[13], data[13] = (0, 13, 2*Minv[2,4])
            row[14], col[14], data[14] = (0, 14, 2*Minv[3,4])
            return coo_matrix((data, (row, col)), shape=(1,15))

In [ ]:
def compute_FIM(numRes, FIMSet, obj="A", fix=False, budget=100, additional_options={}):
    
    m = pyo.ConcreteModel()
    
    m.NumRes = pyo.Set(initialize=range(numRes))
    m.DimFIM = pyo.Set(initialize=range(len(FIMSet[0])))
    print(m.DimFIM)
    
    # set up y and cov y 
    #m.y = pyo.Var(m.NumRes, initialize=0.1, bounds=(0,1), within = pyo.NonNegativeReals)
    def identity(m, a, b):
        return 1 if a==b else 0

    m.cov_y = pyo.Var(m.NumRes, m.NumRes, initialize=identity, bounds=(0,1), within=pyo.NonNegativeReals)
    
    if fix:
        m.cov_y.fix()
    
    # set up FIM
    m.TotalFIM = pyo.Var(m.DimFIM, m.DimFIM, initialize=1)

    # compute FIM
    def eval_fim(m, a, b):
        return m.TotalFIM[a,b] == sum(m.cov_y[i,j]*FIMSet[i*numRes+j][a][b] for i in range(numRes) for j in range(numRes))
    
    
    # y and conv y constraints
    def y_covy1(m, a, b):
        return m.cov_y[a, b] <= m.cov_y[a, a]
    
    def y_covy2(m, a, b):
        return m.cov_y[a, b] <= m.cov_y[b, b]
    
    def y_covy3(m, a, b):
        return m.cov_y[a, b] >= m.cov_y[a, a] + m.cov_y[b, b] - 1
    
    def y_covy4(m, a, b):
        '''
        Avoid duplication
        '''
        if a>b:
            return m.cov_y[a, b] == m.cov_y[b, a]
        else:
            return pyo.Constraint.Skip
    
    def cost_compute(m):
        return m.cost == sum(m.cov_y[i,i] for i in range(numRes))
    
    def cost_limit(m):
        return m.cost <= budget
    
    # set up Design criterion
    def ComputeTrace(m):
        sum_x = sum(m.TotalFIM[j,j] for j in m.DimFIM)
        return sum_x
    
    
    def ComputeDet(m):
        """Calculate determinant. Can be applied to FIM of any size.
        det(A) = sum_{\sigma \in \S_n} (sgn(\sigma) * \Prod_{i=1}^n a_{i,\sigma_i})
        Use permutation() to get permutations, sgn() to get signature
        """

        r_list = list(range(len(m.DimFIM)))
        # get all permutations
        object_p = permutations(r_list)
        list_p = list(object_p)

        # generate a name_order to iterate \sigma_i
        det_perm = 0
        for i in range(len(list_p)):
            name_order = []
            x_order = list_p[i]
            # sigma_i is the value in the i-th position after the reordering \sigma
            for x in range(len(x_order)):
                for y, element in enumerate(m.DimFIM):
                    if x_order[x] == y:
                        name_order.append(element)

            det_perm += sgn(list_p[i]) * (m.TotalFIM[0,name_order[0]] *m.TotalFIM[1,name_order[1]]*m.TotalFIM[2,name_order[2]]*m.TotalFIM[3,name_order[3]]*m.TotalFIM[4,name_order[4]])

        return det_perm
    
    
    # add constraints
    m.TotalFIM_con = pyo.Constraint(m.DimFIM, m.DimFIM, rule=eval_fim)
    
    if not fix:
        m.cov1 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy1)
        m.cov2 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy2)
        m.cov3 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy3)
        m.cov4 = pyo.Constraint(m.NumRes, m.NumRes, rule=y_covy4)
        
        m.cost = pyo.Var(initialize=budget)
        m.cost_compute = pyo.Constraint(rule=cost_compute)
        m.budget_limit = pyo.Constraint(rule=cost_limit)
        
    
    
    # set objective 
    if obj == "A":
        m.Obj = pyo.Objective(rule=ComputeTrace, sense=pyo.maximize)    
    elif obj =="D":
        # create a block to store the external grey box model
        m.my_block = ExternalGreyBoxBlock(external_model=LogDetModel())
    
        # add objective to maximize log det
        m.obj = pyo.Objective(expr=m.my_block.outputs['log_det'], sense=pyo.maximize)
        
        solver = pyo.SolverFactory('cyipopt')

        #solver.config.options['hessian_approximation'] = 'limited-memory'

        for k,v in additional_options.items():
            solver.config.options[k] = v

        results = solver.solve(m, tee=True)
        pyo.assert_optimal_termination(results)
    
    return m 

In [ ]:
total_mea = 14*22

In [ ]:
mod = compute_FIM(total_mea, calculator.fim_collection, obj="D", fix=False, budget=310)

#solver = pyo.SolverFactory('ipopt')
#solver.options['linear_solver'] = 'ma57'
#solver.solve(mod, tee=True)

In [ ]:
print("a = ",mod.my_block.inputs['a'].value)
print("b = ",mod.my_block.inputs['b'].value)
print("obj= ", pyo.value(mod.obj))

In [ ]:
print(pyo.value(mod.Obj))

In [ ]:
### FIM
fim_result = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        fim_result[i,j] = pyo.value(mod.TotalFIM[i,j])
        
print(pyo.value(mod.TotalFIM[0,0]))
        
print(fim_result)
print('trace:', np.trace(fim_result))
print('det:', np.linalg.det(fim_result))

In [ ]:
ans_y = np.zeros((total_mea,total_mea))


#for i in range(70):
    #print('Measurement', i, ':', pyo.value(mod.cov_y[i,i]))
    
for i in range(total_mea):
    for j in range(i, total_mea):
        cov = pyo.value(mod.cov_y[i,j])
        ans_y[i,j] = cov 
        ans_y[j,i] = pyo.value(mod.cov_y[j,i])
        
        if abs(pyo.value(mod.cov_y[i,j]) - pyo.value(mod.cov_y[j,i])) > 0.01:
            print('Wrong symmetry:', i, j, pyo.value(mod.cov_y[i,j]), pyo.value(mod.cov_y[j,i]))
        
        if abs(cov- min(pyo.value(mod.cov_y[i,i]), pyo.value(mod.cov_y[j,j]))) > 0.01:
            print('Wrong computation for covariance y:', i, j, pyo.value(mod.cov_y[i,i]), pyo.value(mod.cov_y[j,j]), cov)
#print(pyo.value(mod.cov_y[5,8]))
    
print(pyo.value(mod.cost))

In [ ]:
print(ans_y)

figure = plt.figure()
axes = figure.add_subplot(111)

caxes = axes.matshow(ans_y, interpolation='nearest')
figure.colorbar(caxes)

plt.show()

In [ ]:
sol_y = np.asarray([ans_y[i,i] for i in range(total_mea)])

sol_y = np.reshape(sol_y, (14,5))
np.around(sol_y)
print(sol_y)

In [ ]:
import matplotlib

def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw=None, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Parameters
    ----------
    data
        A 2D numpy array of shape (M, N).
    row_labels
        A list or array of length M with the labels for the rows.
    col_labels
        A list or array of length N with the labels for the columns.
    ax
        A `matplotlib.axes.Axes` instance to which the heatmap is plotted.  If
        not provided, use current axes or create a new one.  Optional.
    cbar_kw
        A dictionary with arguments to `matplotlib.Figure.colorbar`.  Optional.
    cbarlabel
        The label for the colorbar.  Optional.
    **kwargs
        All other arguments are forwarded to `imshow`.
    """

    if ax is None:
        ax = plt.gca()

    if cbar_kw is None:
        cbar_kw = {}

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # Show all ticks and label them with the respective list entries.
    ax.set_xticks(np.arange(data.shape[1]), labels=col_labels)
    ax.set_yticks(np.arange(data.shape[0]), labels=row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=True, bottom=False,
                   labeltop=True, labelbottom=False)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=-30, ha="right",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    ax.spines[:].set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=("black", "white"),
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Parameters
    ----------
    im
        The AxesImage to be labeled.
    data
        Data used to annotate.  If None, the image's data is used.  Optional.
    valfmt
        The format of the annotations inside the heatmap.  This should either
        use the string format method, e.g. "$ {x:.2f}", or be a
        `matplotlib.ticker.Formatter`.  Optional.
    textcolors
        A pair of colors.  The first is used for values below a threshold,
        the second for those above.  Optional.
    threshold
        Value in data units according to which the colors from textcolors are
        applied.  If None (the default) uses the middle of the colormap as
        separation.  Optional.
    **kwargs
        All other arguments are forwarded to each call to `text` used to create
        the text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)
        
        
    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i, j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
measure_name = ['Ads.gas_inlet.F', 'Ads.gas_outlet.F', 'Ads.gas_outlet.T', 'Ads.gas_outlet.z("CO2")', 'Des.gas_inlet.F', 'Des.gas_outlet.F', 'Des.gas_outlet.T', 'Des.gas_outlet.z("CO2")', 'Ads.T_g.Value(19,10)', 'Ads.T_g.Value(23,10)', 'Ads.T_g.Value(28,10)', 'Ads.z("CO2",19,10)', 'Ads.z("CO2",23,10)', 'Ads.z("CO2",28,10)']

time_set = ['500', '1000', '1500', '2000', '2200']
#time_set = np.linspace(100, 2200, 22)
time_set = [str(i) for i in time_set]

fig, ax = plt.subplots(figsize=(16,8))

im, cbar = heatmap(sol_y, measure_name, time_set, ax=ax,
                   cmap="YlGn", cbarlabel="weight")
texts = annotate_heatmap(im, valfmt="{x:.1f}")

fig.tight_layout()
plt.show()

In [ ]:
def plot_data(budget, a_cri, d_cri, design_cri_label):
    sense = []

    #plt.scatter(budget, np.log10(a_cri), s=50, color='red', label='A-optimality')
    plt.scatter(budget, np.log10(d_cri), marker='v', s=50, color='blue', label='D-optimality')
    plt.xlabel('Budget')
    plt.ylabel('log10(' + design_cri_label + ')')
    plt.legend()

    plt.show()

In [ ]:
a_set = [869.53, 1496.64, 1538.93, 1563.76, 1567.70, 1569.73, 1571.23]
d_set = [140.57, 2852.58, 11349.32, 23168.51, 30715.88, 34988.78, 36029.47]

plot_data([10,20,30,40,50,60,70], a_set, d_set, 'Determinant')